In [2]:
import xgboost

In [5]:
import pickle as pkl

In [7]:
import os
os.chdir("..")

In [8]:
# unzip model
import tarfile
tar = tarfile.open('sagemaker/model.tar.gz')
tar.extractall()
tar.close() 

modelv0 = pkl.load(open("xgboost-model", "rb"))

[23:44:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
modelv0.predict

In [1]:
import os
import json
import pandas as pd

In [2]:
os.chdir("..")

In [4]:
from calypso.feature_engineering import Engineering
from calypso.model_selection import Training

In [5]:
feature_params = json.load(open("settings/dataprep.json", "r"))

In [10]:
model_params = json.load(open("settings/train.json", "r"))

In [6]:
train = pd.read_csv("data/train.csv")

In [7]:
data = Engineering(train)

In [8]:
y = data.target("SalePrice")

In [9]:
X = data.feature(feature_params)

In [11]:
model = Training(X, y, model_params)

In [12]:
model.train_test_split()

In [13]:
model.gridsearch()

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  30 out of  30 | elapsed:    9.9s finished


GridSearchCV(estimator=Pipeline(steps=[('estimator', RandomForestRegressor())]),
             n_jobs=6,
             param_grid=[{'estimator': [RandomForestRegressor(random_state=1903)],
                          'estimator__ccp_alpha': [0],
                          'estimator__max_features': ['auto'],
                          'estimator__min_samples_leaf': [1, 3],
                          'estimator__min_samples_split': [20],
                          'estimator__n_estimators': [200, 300]},
                         {'estimator': [GradientBoostingRegressor(loss='huber',
                                                                  n_estimators=200,
                                                                  random_state=1903)],
                          'estimator__learning_rate': [0.1],
                          'estimator__n_estimators': [200, 300]}],
             scoring='neg_mean_squared_error', verbose=2)

In [16]:
model.save()

In [14]:
best_model = model.best()

In [ ]:
[mse_train, mse_valid] = model.validate(X_valid, y_valid)